In [1]:
import cv2
import numpy as np
from retinaface import RetinaFace
import face_recognition
import os

# Directory containing known faces
KNOWN_FACES_DIR = r"C:\Users\vamshi\OneDrive\Desktop\retina\Training_images"
TOLERANCE = 0.6  # Recognition tolerance
FRAME_THICKNESS = 3  # Thickness of bounding box
FONT_THICKNESS = 2  # Thickness of text
MODEL = "cnn"  # The model can be "hog" or "cnn" for detection

# Load known faces and their encodings
def load_known_faces():
    known_faces = []
    known_names = []

    for name in os.listdir(KNOWN_FACES_DIR):
        for filename in os.listdir(f"{KNOWN_FACES_DIR}\{name}"):
            image = face_recognition.load_image_file(f"{KNOWN_FACES_DIR}\{name}\{filename}")
            encodings = face_recognition.face_encodings(image)
            if encodings:
                known_faces.append(encodings[0])
                known_names.append(name)
    print("loaded")
    return known_faces, known_names

# Draw bounding boxes and names
def draw_faces(frame, faces, names):
    for (face, name) in zip(faces, names):
        # Draw a rectangle around the face
        cv2.rectangle(frame, (face[0], face[1]), (face[2], face[3]), (0, 255, 0), FRAME_THICKNESS)

        # Draw a label with a name below the face
        y_position = face[1] - 15 if face[1] - 15 > 15 else face[1] + 15
        cv2.rectangle(frame, (face[0], face[1]), (face[2], face[1] - 35), (0, 255, 0), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (face[0] + 6, y_position), font, 0.8, (255, 255, 255), FONT_THICKNESS)

# Recognize faces from live video
def recognize_faces_from_live_video(known_faces, known_names):
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Detect faces with RetinaFace
        faces = RetinaFace.detect_faces(frame)
        print("frame recorded")
        # Prepare face encodings for recognition
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = []
        face_names = []

        for face_key in faces.keys():
            facial_area = faces[face_key]["facial_area"]

            # Append detected face locations for recognition
            face_locations.append(facial_area)

            # Convert RetinaFace coordinates to dlib format
            top, right, bottom, left = facial_area[1], facial_area[2], facial_area[3], facial_area[0]
            # Encode the face using face_recognition
            encodings = face_recognition.face_encodings(frame_rgb, [(top, right, bottom, left)])

            for encoding in encodings:
                # Compare with known faces
                matches = face_recognition.compare_faces(known_faces, encoding, TOLERANCE)
                name = "Unknown"

                # Find the shortest distance to a known face
                face_distances = face_recognition.face_distance(known_faces, encoding)
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = known_names[best_match_index]
                print(name," Is identified")
                face_names.append(name)

        # Draw the results
        draw_faces(frame, face_locations, face_names)

        # Display the frame
        cv2.imshow('Live Video', frame)
        
        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    # Load known faces
    known_faces, known_names = load_known_faces()
    print("Loaded Done")
    # print(known_faces)
    print(known_names)
    recognize_faces_from_live_video(known_faces, known_names)



loaded
Loaded Done
['shreyesh', 'vamshi']
frame recorded
Unknown  Is identified
frame recorded
Unknown  Is identified
frame recorded
Unknown  Is identified


KeyboardInterrupt: 

In [3]:
import cv2
import numpy as np
from retinaface import RetinaFace
import face_recognition
import os
import csv
from datetime import datetime, timedelta
import threading
KNOWN_FACES_DIR = r"C:\Users\vamshi\OneDrive\Desktop\retina\Training_images"
TOLERANCE = 0.6  # Recognition tolerance
FRAME_THICKNESS = 3  # Thickness of bounding box
FONT_THICKNESS = 2  # Thickness of text
MODEL = "cnn"  # The model can be "hog" or "cnn" for detection
def load_known_faces():
    known_faces = []
    known_names = []
    for name in os.listdir(KNOWN_FACES_DIR):
        for filename in os.listdir(f"{KNOWN_FACES_DIR}/{name}"):
            image = face_recognition.load_image_file(f"{KNOWN_FACES_DIR}/{name}/{filename}")
            encodings = face_recognition.face_encodings(image)
            if encodings:
                known_faces.append(encodings[0])
                known_names.append(name)
    print("Loaded known faces")
    return known_faces, known_names
def draw_faces(frame, faces, names):
    for (face, name) in zip(faces, names):
        cv2.rectangle(frame, (face[0], face[1]), (face[2], face[3]), (0, 255, 0), FRAME_THICKNESS)
        y_position = face[1] - 15 if face[1] - 15 > 15 else face[1] + 15
        cv2.rectangle(frame, (face[0], face[1]), (face[2], face[1] - 35), (0, 255, 0), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (face[0] + 6, y_position), font, 0.8, (255, 255, 255), FONT_THICKNESS)
        
def recognize_faces_from_camera(camera_id, known_faces, known_names, tracking_dict):
    cap = cv2.VideoCapture(camera_id)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        faces = RetinaFace.detect_faces(frame)
        print(f"Frame recorded from Camera {camera_id}")
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = []
        face_names = []
        current_faces = set() 
        for face_key in faces.keys():
            facial_area = faces[face_key]["facial_area"]
            face_locations.append(facial_area)
            current_faces.add(facial_area)
            top, right, bottom, left = facial_area[1], facial_area[2], facial_area[3], facial_area[0]
            encodings = face_recognition.face_encodings(frame_rgb, [(top, right, bottom, left)])
            for encoding in encodings:
                matches = face_recognition.compare_faces(known_faces, encoding, TOLERANCE)
                name = "Unknown"
                face_distances = face_recognition.face_distance(known_faces, encoding)
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = known_names[best_match_index]
                face_names.append(name)
                if name not in tracking_dict['seen']:
                    if camera_id == 0:
                        tracking_dict['entry_writer'].writerow([name, datetime.now().strftime('%Y-%m-%d %H:%M:%S')])
                        tracking_dict['seen'][name] = datetime.now()
                        tracking_dict['exit_seen'][name] = False
                    elif camera_id == 1:
                        if not tracking_dict['exit_seen'].get(name, False):
                            tracking_dict['exit_writer'].writerow([name, datetime.now().strftime('%Y-%m-%d %H:%M:%S')])
                            tracking_dict['exit_seen'][name] = True
        if camera_id == 0:
            absent_names = set(tracking_dict['seen'].keys()) - set(face_names)
            for name in absent_names:
                if not tracking_dict['exit_seen'][name]:
                    if datetime.now() - tracking_dict['seen'][name] >= timedelta(minutes=2):
                        tracking_dict['exit_writer'].writerow([name, datetime.now().strftime('%Y-%m-%d %H:%M:%S')])
                        tracking_dict['exit_seen'][name] = True
        draw_faces(frame, face_locations, face_names)
        cv2.imshow(f'Camera {camera_id}', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
def main():
    known_faces, known_names = load_known_faces()
    print("Loaded Done")
    print(known_names)
    with open(r'C:\Users\vamshi\OneDrive\Desktop\retina\attendance.csv', mode='a', newline='') as file:
        entry_writer = csv.writer(file)
        exit_writer = csv.writer(file)
        tracking_dict = {
            'entry_writer': entry_writer,
            'exit_writer': exit_writer,
            'seen': {},  
            'exit_seen': {} 
        }
        camera_ids = [0, 1]  
        threads = []
        for camera_id in camera_ids:
            thread = threading.Thread(target=recognize_faces_from_camera, args=(camera_id, known_faces, known_names, tracking_dict))
            thread.start()
            threads.append(thread)
        for thread in threads:
            thread.join()
if __name__ == "__main__":
    main()


Loaded known faces
Loaded Done
['shreyesh', 'vamshi']


In [ ]:
import cv2
import numpy as np
from retinaface import RetinaFace
import face_recognition
import os
import csv
from datetime import datetime, timedelta

# Directory containing known faces
KNOWN_FACES_DIR = r"C:\attendance2\images"
TOLERANCE = 0.6  # Recognition tolerance
FRAME_THICKNESS = 3  # Thickness of bounding box
FONT_THICKNESS = 2  # Thickness of text
MODEL = "cnn"  # The model can be "hog" or "cnn" for detection

# Load known faces and their encodings
def load_known_faces():
    known_faces = []
    known_names = []

    for name in os.listdir(KNOWN_FACES_DIR):
        for filename in os.listdir(f"{KNOWN_FACES_DIR}/{name}"):
            image = face_recognition.load_image_file(f"{KNOWN_FACES_DIR}/{name}/{filename}")
            encodings = face_recognition.face_encodings(image)
            if encodings:
                known_faces.append(encodings[0])
                known_names.append(name)
    print("loaded")
    return known_faces, known_names

# Draw bounding boxes and names
def draw_faces(frame, faces, names):
    for (face, name) in zip(faces, names):
        # Draw a rectangle around the face
        cv2.rectangle(frame, (face[0], face[1]), (face[2], face[3]), (0, 255, 0), FRAME_THICKNESS)

        # Draw a label with a name below the face
        y_position = face[1] - 15 if face[1] - 15 > 15 else face[1] + 15
        cv2.rectangle(frame, (face[0], face[1]), (face[2], face[1] - 35), (0, 255, 0), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (face[0] + 6, y_position), font, 0.8, (255, 255, 255), FONT_THICKNESS)

# Recognize faces from live video and log to CSV
def recognize_faces_from_live_video(known_faces, known_names):
    cap = cv2.VideoCapture(0)
    
    # Open a CSV file for writing
    with open('C:/attendance2/attendance_log.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Name', 'Timestamp'])  # Header row
        
        last_record_times = {}  # Dictionary to track last record time for each person
        frame_count = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Process every nth frame to reduce the load
            frame_count += 1
            if frame_count % 5 != 0:  # Change the modulo value to adjust the frame rate
                continue

            # Detect faces with RetinaFace
            faces = RetinaFace.detect_faces(frame)
            print("frame recorded")
            # Prepare face encodings for recognition
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            face_locations = []
            face_names = []

            for face_key in faces.keys():
                facial_area = faces[face_key]["facial_area"]

                # Append detected face locations for recognition
                face_locations.append(facial_area)

                # Convert RetinaFace coordinates to dlib format
                top, right, bottom, left = facial_area[1], facial_area[2], facial_area[3], facial_area[0]
                # Encode the face using face_recognition
                encodings = face_recognition.face_encodings(frame_rgb, [(top, right, bottom, left)])

                for encoding in encodings:
                    # Compare with known faces
                    matches = face_recognition.compare_faces(known_faces, encoding, TOLERANCE)
                    name = "Unknown"

                    # Find the shortest distance to a known face
                    face_distances = face_recognition.face_distance(known_faces, encoding)
                    best_match_index = np.argmin(face_distances)
                    if matches[best_match_index]:
                        name = known_names[best_match_index]
                    print(name, "is identified")
                    face_names.append(name)

            # Log detected names and timestamps if 2 minutes have passed
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            for name in face_names:
                if name != "Unknown":
                    if name not in last_record_times or (datetime.now() - last_record_times[name]) >= timedelta(minutes=2):
                        writer.writerow([name, timestamp])
                        last_record_times[name] = datetime.now()  # Update the last record time for this person

            # Draw the results
            draw_faces(frame, face_locations, face_names)

            # Display the frame
            cv2.imshow('Live Video', frame)
            
            # Press 'q' to exit
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    # Load known faces
    known_faces, known_names = load_known_faces()
    print("Loaded Done")
    print(known_names)
    recognize_faces_from_live_video(known_faces, known_names)


In [ ]:
import cv2
import numpy as np
from retinaface import RetinaFace
import face_recognition
import os
import pandas as pd
from datetime import datetime, timedelta

# Directory containing known faces
KNOWN_FACES_DIR = r"C:\Users\vamshi\OneDrive\Desktop\retina\Training_images"
TOLERANCE = 0.6
FRAME_THICKNESS = 3
FONT_THICKNESS = 2
MODEL = "cnn"

# Constants for periods and attendance calculation
PERIOD_TIMES = [
    (9, 10),   # Period 1: 9-10 am
    (10, 11),  # Period 2: 10-11 am
    (11, 12),  # Period 3: 11-12 pm
    (13, 14),  # Period 4: 1-2 pm
    (14, 15),  # Period 5: 2-3 pm
    (15, 16)   # Period 6: 3-4 pm
]
TOTAL_PERIODS = 6

# Load known faces and their encodings
def load_known_faces():
    known_faces = []
    known_names = []

    for name in os.listdir(KNOWN_FACES_DIR):
        for filename in os.listdir(f"{KNOWN_FACES_DIR}/{name}"):
            image = face_recognition.load_image_file(f"{KNOWN_FACES_DIR}/{name}/{filename}")
            encodings = face_recognition.face_encodings(image)
            if encodings:
                known_faces.append(encodings[0])
                known_names.append(name)
    print("Loaded known faces.")
    return known_faces, known_names

# Initialize or update attendance file with columns for each period and Total and Percentage
def initialize_attendance_file(known_names):
    attendance_file = r'C:\Users\vamshi\OneDrive\Desktop\retina\Attend.xlsx'
    if not os.path.exists(attendance_file):
        columns = ['Name'] + [f'Period {i+1}' for i in range(TOTAL_PERIODS)] + ['Total', 'Percentage']
        attendance_data = pd.DataFrame(columns=columns)
        attendance_data['Name'] = known_names
        attendance_data.fillna(0, inplace=True)  # Set initial attendance to 0
        attendance_data.to_excel(attendance_file, index=False)
    else:
        attendance_data = pd.read_excel(attendance_file, engine='openpyxl')
    return attendance_data

# Determine current period based on the current time
def get_period_index(current_time):
    for i, (start, end) in enumerate(PERIOD_TIMES):
        if start <= current_time.hour < end:
            return i  # Return the period index (0 for Period 1, etc.)
    return None  # Return None if outside period times

# Update attendance for each period based on entry time
def update_attendance(name, timestamp, attendance_data):
    attendance_file = r'C:\Users\vamshi\OneDrive\Desktop\retina\Attend.xlsx'
    current_time = datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
    period_index = get_period_index(current_time)
    
    if period_index is not None:
        period_col = f'Period {period_index + 1}'
        
        # Increment the attendance count for the detected period
        attendance_data.loc[attendance_data['Name'] == name, period_col] += 1
        
        # Update total and recalculate attendance percentage
        attendance_data['Total'] = attendance_data[[f'Period {i+1}' for i in range(TOTAL_PERIODS)]].sum(axis=1)
        prev_percentage = attendance_data.loc[attendance_data['Name'] == name, 'Percentage'].iloc[0]
        classes_today = attendance_data.loc[attendance_data['Name'] == name, [f'Period {i+1}' for i in range(TOTAL_PERIODS)]].sum()
        new_percentage = (prev_percentage + (classes_today / TOTAL_PERIODS)) / 2
        attendance_data.loc[attendance_data['Name'] == name, 'Percentage'] = new_percentage
        
        # Save updated attendance data to Excel
        attendance_data.to_excel(attendance_file, index=False)
        print(f"Updated attendance for {name} at {timestamp} for {period_col}")

# Recognize faces from entry and exit cameras and process attendance
def recognize_faces_multi_camera(known_faces, known_names):
    cap_entry = cv2.VideoCapture(0)  # Entry camera
    cap_exit = cv2.VideoCapture(1)   # Exit camera

    attendance_data = initialize_attendance_file(known_names)
    global last_entry_times
    last_entry_times = {}  # Dictionary to track last entry times for each person

    while True:
        # Read frames from both entry and exit cameras
        ret_entry, frame_entry = cap_entry.read()
        ret_exit, frame_exit = cap_exit.read()
        if not ret_entry or not ret_exit:
            break

        # Process entry camera
        process_frame(frame_entry, 'entry', known_faces, known_names, attendance_data)

        # Process exit camera
        process_frame(frame_exit, 'exit', known_faces, known_names, attendance_data)

        # Display frames for entry and exit cameras
        cv2.imshow('Entry Camera', frame_entry)
        cv2.imshow('Exit Camera', frame_exit)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap_entry.release()
    cap_exit.release()
    cv2.destroyAllWindows()

# Process frames for entry or exit events
def process_frame(frame, event_type, known_faces, known_names, attendance_data):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = RetinaFace.detect_faces(frame)
    face_locations = []
    face_names = []

    for face_key in faces.keys():
        facial_area = faces[face_key]["facial_area"]
        top, right, bottom, left = facial_area[1], facial_area[2], facial_area[3], facial_area[0]
        encodings = face_recognition.face_encodings(frame_rgb, [(top, right, bottom, left)])

        for encoding in encodings:
            matches = face_recognition.compare_faces(known_faces, encoding, TOLERANCE)
            name = "Unknown"
            face_distances = face_recognition.face_distance(known_faces, encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_names[best_match_index]
            face_names.append(name)

    # Log detected names and timestamps
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    for name in face_names:
        if name != "Unknown":
            update_attendance(name, timestamp, attendance_data)
            print(f"{event_type.capitalize()} detected for {name} at {timestamp}")

# Entry point
if __name__ == "__main__":
    known_faces, known_names = load_known_faces()
    recognize_faces_multi_camera(known_faces, known_names)



Loaded known faces.
Updated attendance for vamshi at 2024-10-30 13:34:32 for Period 4
Entry detected for vamshi at 2024-10-30 13:34:32
Updated attendance for vamshi at 2024-10-30 13:35:00 for Period 4
Entry detected for vamshi at 2024-10-30 13:35:00


In [ ]:
import cv2
import numpy as np
from retinaface import RetinaFace
import face_recognition
import os
import pandas as pd
from datetime import datetime, timedelta

# Directory containing known faces
KNOWN_FACES_DIR = r"C:\Users\vamshi\OneDrive\Desktop\retina\Training_images"
TOLERANCE = 0.6
FRAME_THICKNESS = 3
FONT_THICKNESS = 2
MODEL = "cnn"

# Constants for periods and attendance calculation
PERIOD_TIMES = [
    (9, 10),   # Period 1: 9-10 am
    (10, 11),  # Period 2: 10-11 am
    (11, 12),  # Period 3: 11-12 pm
    (13, 14),  # Period 4: 1-2 pm
    (14, 15),  # Period 5: 2-3 pm
    (15, 16)   # Period 6: 3-4 pm
]
TOTAL_PERIODS = 6
MINIMUM_DURATION = 2  # in minutes to log attendance for the period

# Load known faces and their encodings
def load_known_faces():
    known_faces = []
    known_names = []

    for name in os.listdir(KNOWN_FACES_DIR):
        for filename in os.listdir(f"{KNOWN_FACES_DIR}/{name}"):
            image = face_recognition.load_image_file(f"{KNOWN_FACES_DIR}/{name}/{filename}")
            encodings = face_recognition.face_encodings(image)
            if encodings:
                known_faces.append(encodings[0])
                known_names.append(name)
    print("Loaded known faces.")
    return known_faces, known_names

# Initialize or update attendance file with columns for each period and Total and Percentage
def initialize_attendance_file(known_names):
    attendance_file = r'C:\Users\vamshi\OneDrive\Desktop\retina\Attend.xlsx'
    if not os.path.exists(attendance_file):
        columns = ['Name'] + [f'Period {i+1}' for i in range(TOTAL_PERIODS)] + ['Total', 'Percentage']
        attendance_data = pd.DataFrame(columns=columns)
        attendance_data['Name'] = known_names
        attendance_data.fillna(0, inplace=True)  # Set initial attendance to 0
        attendance_data.to_excel(attendance_file, index=False)
    else:
        attendance_data = pd.read_excel(attendance_file, engine='openpyxl')
    return attendance_data

# Determine current period based on the current time
def get_period_index(current_time):
    for i, (start, end) in enumerate(PERIOD_TIMES):
        if start <= current_time.hour < end:
            return i  # Return the period index (0 for Period 1, etc.)
    return None  # Return None if outside period times

# Update attendance for each period based on entry and exit times
def update_attendance(name, timestamp, event_type, attendance_data, last_entry_times):
    attendance_file = r'C:\Users\vamshi\OneDrive\Desktop\retina\Attend.xlsx'
    current_time = datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
    period_index = get_period_index(current_time)
    
    if period_index is not None:
        period_col = f'Period {period_index + 1}'
        
        if event_type == 'entry':
            last_entry_times[name] = current_time  # Store entry time
        elif event_type == 'exit' and name in last_entry_times:
            entry_time = last_entry_times.pop(name, None)  # Retrieve and remove the entry time
            duration = (current_time - entry_time).total_seconds() / 60  # Convert to minutes

            # Log attendance only if the duration is at least 45 minutes
            if duration >= MINIMUM_DURATION:
                attendance_data.loc[attendance_data['Name'] == name, period_col] += 1
                
                # Update total and recalculate attendance percentage
                attendance_data['Total'] = attendance_data[[f'Period {i+1}' for i in range(TOTAL_PERIODS)]].sum(axis=1)
                prev_percentage = attendance_data.loc[attendance_data['Name'] == name, 'Percentage'].iloc[0]
                classes_today = attendance_data.loc[attendance_data['Name'] == name, [f'Period {i+1}' for i in range(TOTAL_PERIODS)]].sum()
                new_percentage = (prev_percentage + (classes_today / TOTAL_PERIODS)) / 2
                attendance_data.loc[attendance_data['Name'] == name, 'Percentage'] = new_percentage
                
                # Save updated attendance data to Excel
                attendance_data.to_excel(attendance_file, index=False)
                print(f"Updated attendance for {name} for {period_col} (duration: {duration} mins)")

# Recognize faces from entry and exit cameras and process attendance
def recognize_faces_multi_camera(known_faces, known_names):
    cap_entry = cv2.VideoCapture(0)  # Entry camera
    cap_exit = cv2.VideoCapture(1)   # Exit camera

    attendance_data = initialize_attendance_file(known_names)
    last_entry_times = {}  # Dictionary to track last entry times for each person

    while True:
        # Read frames from both entry and exit cameras
        ret_entry, frame_entry = cap_entry.read()
        ret_exit, frame_exit = cap_exit.read()
        if not ret_entry or not ret_exit:
            break

        # Process entry camera
        process_frame(frame_entry, 'entry', known_faces, known_names, attendance_data, last_entry_times)

        # Process exit camera
        process_frame(frame_exit, 'exit', known_faces, known_names, attendance_data, last_entry_times)

        # Display frames for entry and exit cameras
        cv2.imshow('Entry Camera', frame_entry)
        cv2.imshow('Exit Camera', frame_exit)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap_entry.release()
    cap_exit.release()
    cv2.destroyAllWindows()

# Process frames for entry or exit events
def process_frame(frame, event_type, known_faces, known_names, attendance_data, last_entry_times):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = RetinaFace.detect_faces(frame)
    face_names = []

    for face_key in faces.keys():
        facial_area = faces[face_key]["facial_area"]
        top, right, bottom, left = facial_area[1], facial_area[2], facial_area[3], facial_area[0]
        encodings = face_recognition.face_encodings(frame_rgb, [(top, right, bottom, left)])

        for encoding in encodings:
            matches = face_recognition.compare_faces(known_faces, encoding, TOLERANCE)
            name = "Unknown"
            face_distances = face_recognition.face_distance(known_faces, encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_names[best_match_index]
            face_names.append(name)

    # Log detected names and timestamps
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    for name in face_names:
        if name != "Unknown":
            update_attendance(name, timestamp, event_type, attendance_data, last_entry_times)
            print(f"{event_type.capitalize()} detected for {name} at {timestamp}")

# Entry point
if __name__ == "__main__":
    known_faces, known_names = load_known_faces()
    recognize_faces_multi_camera(known_faces, known_names)



Loaded known faces.
Entry detected for vamshi at 2024-10-30 14:39:29
Updated attendance for vamshi for Period 5 (duration: 2.85 mins)
Exit detected for vamshi at 2024-10-30 14:42:20


In [ ]:
import cv2
import numpy as np
from retinaface import RetinaFace
import face_recognition
import os
import pandas as pd
from datetime import datetime, timedelta

# Directory containing known faces
KNOWN_FACES_DIR = r"C:\Users\vamshi\OneDrive\Desktop\retina\Training_images"
TOLERANCE = 0.6
FRAME_THICKNESS = 3
FONT_THICKNESS = 2
MODEL = "cnn"

# Constants for periods and attendance calculation
PERIOD_TIMES = [
    (9, 10),   # Period 1: 9-10 am
    (10, 11),  # Period 2: 10-11 am
    (11, 12),  # Period 3: 11-12 pm
    (13, 14),  # Period 4: 1-2 pm
    (14, 15),  # Period 5: 2-3 pm
    (15, 16)   # Period 6: 3-4 pm
]
TOTAL_PERIODS = 6
MINIMUM_DURATION = 45  # in minutes to log attendance for the period

# Load known faces and their encodings
def load_known_faces():
    known_faces = []
    known_names = []

    for name in os.listdir(KNOWN_FACES_DIR):
        for filename in os.listdir(f"{KNOWN_FACES_DIR}/{name}"):
            image = face_recognition.load_image_file(f"{KNOWN_FACES_DIR}/{name}/{filename}")
            encodings = face_recognition.face_encodings(image)
            if encodings:
                known_faces.append(encodings[0])
                known_names.append(name)
    print("Loaded known faces.")
    return known_faces, known_names

# Initialize or update attendance file with columns for each period and Total and Percentage
def initialize_attendance_file(known_names):
    attendance_file = r'C:\Users\vamshi\OneDrive\Desktop\retina\Attend.xlsx'
    if not os.path.exists(attendance_file):
        columns = ['Name'] + [f'Period {i+1}' for i in range(TOTAL_PERIODS)] + ['Total', 'Percentage']
        attendance_data = pd.DataFrame(columns=columns)
        attendance_data['Name'] = known_names
        attendance_data.fillna(0, inplace=True)  # Set initial attendance to 0
        attendance_data.to_excel(attendance_file, index=False)
    else:
        attendance_data = pd.read_excel(attendance_file, engine='openpyxl')
    return attendance_data

# Determine current period based on the current time
def get_period_index(current_time):
    for i, (start, end) in enumerate(PERIOD_TIMES):
        if start <= current_time.hour < end:
            return i  # Return the period index (0 for Period 1, etc.)
    return None  # Return None if outside period times

# Update attendance for each period based on entry and exit times
def update_attendance(name, timestamp, event_type, attendance_data, last_entry_times):
    attendance_file = r'C:\Users\vamshi\OneDrive\Desktop\retina\Attend.xlsx'
    current_time = datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
    period_index = get_period_index(current_time)
    
    if period_index is not None:
        period_col = f'Period {period_index + 1}'
        
        if event_type == 'entry':
            last_entry_times[name] = current_time  # Store entry time
        elif event_type == 'exit' and name in last_entry_times:
            entry_time = last_entry_times.pop(name, None)  # Retrieve and remove the entry time
            duration = (current_time - entry_time).total_seconds() / 60  # Convert to minutes

            # Log attendance only if the duration is at least 45 minutes
            if duration >= MINIMUM_DURATION:
                attendance_data.loc[attendance_data['Name'] == name, period_col] += 1

                # Update total attendance count
                attendance_data['Total'] = attendance_data[[f'Period {i + 1}' for i in range(TOTAL_PERIODS)]].sum(axis=1)

                # Update percentage
                total_attended = attendance_data.loc[attendance_data['Name'] == name, 'Total'].values[0]
                attendance_data.loc[attendance_data['Name'] == name, 'Percentage'] = (total_attended / TOTAL_PERIODS) * 100

                # Save updated attendance data to Excel
                attendance_data.to_excel(attendance_file, index=False)
                print(f"Updated attendance for {name} for {period_col} (duration: {duration} mins)")

# Recognize faces from entry and exit cameras and process attendance
def recognize_faces_multi_camera(known_faces, known_names):
    cap_entry = cv2.VideoCapture(0)  # Entry camera
    cap_exit = cv2.VideoCapture(1)   # Exit camera

    attendance_data = initialize_attendance_file(known_names)
    last_entry_times = {}  # Dictionary to track last entry times for each person

    while True:
        # Read frames from both entry and exit cameras
        ret_entry, frame_entry = cap_entry.read()
        ret_exit, frame_exit = cap_exit.read()
        if not ret_entry or not ret_exit:
            break

        # Process entry camera
        process_frame(frame_entry, 'entry', known_faces, known_names, attendance_data, last_entry_times)

        # Process exit camera
        process_frame(frame_exit, 'exit', known_faces, known_names, attendance_data, last_entry_times)

        # Display frames for entry and exit cameras
        cv2.imshow('Entry Camera', frame_entry)
        cv2.imshow('Exit Camera', frame_exit)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap_entry.release()
    cap_exit.release()
    cv2.destroyAllWindows()

# Process frames for entry or exit events
def process_frame(frame, event_type, known_faces, known_names, attendance_data, last_entry_times):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = RetinaFace.detect_faces(frame)
    face_names = []

    for face_key in faces.keys():
        facial_area = faces[face_key]["facial_area"]
        top, right, bottom, left = facial_area[1], facial_area[2], facial_area[3], facial_area[0]
        encodings = face_recognition.face_encodings(frame_rgb, [(top, right, bottom, left)])

        for encoding in encodings:
            matches = face_recognition.compare_faces(known_faces, encoding, TOLERANCE)
            name = "Unknown"
            face_distances = face_recognition.face_distance(known_faces, encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_names[best_match_index]
            face_names.append(name)

    # Log detected names and timestamps
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    for name in face_names:
        if name != "Unknown":
            update_attendance(name, timestamp, event_type, attendance_data, last_entry_times)
            print(f"{event_type.capitalize()} detected for {name} at {timestamp}")

# Entry point
if __name__ == "__main__":
    known_faces, known_names = load_known_faces()
    recognize_faces_multi_camera(known_faces, known_names)


In [ ]:
import cv2
import numpy as np
from retinaface import RetinaFace
import face_recognition
import os
import pandas as pd
from datetime import datetime, timedelta

# Directory containing known faces
KNOWN_FACES_DIR = r"C:\Users\vamshi\OneDrive\Desktop\retina\Training_images"
TOLERANCE = 0.6
FRAME_THICKNESS = 3
FONT_THICKNESS = 2
MODEL = "cnn"

# Constants for periods and attendance calculation
PERIOD_TIMES = [
    (9, 10),   # Period 1: 9-10 am
    (10, 11),  # Period 2: 10-11 am
    (11, 12),  # Period 3: 11-12 pm
    (13, 14),  # Period 4: 1-2 pm
    (14, 15),  # Period 5: 2-3 pm
    (0, 1)   # Period 6: 3-4 pm
]
TOTAL_PERIODS = len(PERIOD_TIMES)
MINIMUM_DURATION = 1  # Minimum duration in minutes to log attendance

# Load known faces and their encodings
def load_known_faces():
    known_faces = []
    known_names = []

    for name in os.listdir(KNOWN_FACES_DIR):
        for filename in os.listdir(f"{KNOWN_FACES_DIR}/{name}"):
            image = face_recognition.load_image_file(f"{KNOWN_FACES_DIR}/{name}/{filename}")
            encodings = face_recognition.face_encodings(image)
            if encodings:
                known_faces.append(encodings[0])
                known_names.append(name)
    print("Loaded known faces.")
    return known_faces, known_names

  # Initialize or update attendance file with columns for each period and Total and Percentage
def initialize_attendance_file(known_names):
    attendance_file = r'C:\Users\vamshi\OneDrive\Desktop\retina\Attend.xlsx'
    if not os.path.exists(attendance_file):
        columns = ['Name', 'Date'] + [f'Period {i+1}' for i in range(TOTAL_PERIODS)] + ['Total', 'Percentage']
        attendance_data = pd.DataFrame(columns=columns)
        attendance_data['Name'] = pd.Series(known_names).unique() 
        attendance_data['Percentage'] = 100
        attendance_data['Date'] = datetime.now().date()  # Set today's date
        attendance_data.fillna(0, inplace=True)  # Set initial attendance to 0
        attendance_data.to_excel(attendance_file, index=False)
    else:
        attendance_data = pd.read_excel(attendance_file, engine='openpyxl')
    return attendance_data


# Determine current period based on the current time
def get_period_index(current_time):
    for i, (start, end) in enumerate(PERIOD_TIMES):
        if start <= current_time.hour < end:
            return i  # Return the period index (0 for Period 1, etc.)
    return None  # Return None if outside period times

# Update attendance for each period based on entry and exit times
# Update attendance for each period based on entry and exit times
def update_attendance(name, timestamp, event_type, attendance_data, last_entry_times):
    attendance_file = r'C:\Users\vamshi\OneDrive\Desktop\retina\Attend.xlsx'
    current_time = datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
    period_index = get_period_index(current_time)

    if period_index is not None:
        period_col = f'Period {period_index + 1}'

        if event_type == 'entry':
            last_entry_times[name] = current_time  # Store entry time
        elif event_type == 'exit' and name in last_entry_times:
            entry_time = last_entry_times.pop(name)  # Retrieve and remove the entry time
            duration = (current_time - entry_time).total_seconds() / 60  # Convert to minutes

            # Log attendance only if the duration is at least MINIMUM_DURATION
            if duration >= MINIMUM_DURATION:
                # Increment attendance for the current period
                attendance_data.loc[attendance_data['Name'] == name, period_col] += 1
                
                # Update total attendance
                current_total = attendance_data.loc[attendance_data['Name'] == name, 'Total'].iloc[0]
                new_total = current_total + 1  # Increase total attendance count by 1
                attendance_data.loc[attendance_data['Name'] == name, 'Total'] = new_total
                
                # Update total periods attended
                total_periods_attended = attendance_data.loc[attendance_data['Name'] == name, 'Total'].iloc[0]
                
                # Update the total possible periods (this would need to consider the total number of periods over time)
                # For this example, let's assume TOTAL_PERIODS is constant. If it changes, you need to adjust accordingly.
                total_possible_periods = TOTAL_PERIODS * ((datetime.now().date() - attendance_data['Date'].min().date()).days+1)
                    
                # Calculate the attendance percentage
                attendance_data.loc[attendance_data['Name'] == name, 'Percentage'] = (total_periods_attended / total_possible_periods) * 100
                
                # Save updated attendance data to Excel
                attendance_data.to_excel(attendance_file, index=False)
                print(f"Updated attendance for {name} for {period_col} (duration: {duration:.2f} mins)")



# Recognize faces from entry and exit cameras and process attendance
def recognize_faces_multi_camera(known_faces, known_names):
    cap_entry = cv2.VideoCapture(0)  # Entry camera
    cap_exit = cv2.VideoCapture(1)   # Exit camera

    attendance_data = initialize_attendance_file(known_names)
    last_entry_times = {}  # Dictionary to track last entry times for each person

    while True:
        # Read frames from both entry and exit cameras
        ret_entry, frame_entry = cap_entry.read()
        ret_exit, frame_exit = cap_exit.read()
        if not ret_entry or not ret_exit:
            break

        # Process entry camera
        process_frame(frame_entry, 'entry', known_faces, known_names, attendance_data, last_entry_times)

        # Process exit camera
        process_frame(frame_exit, 'exit', known_faces, known_names, attendance_data, last_entry_times)

        # Display frames for entry and exit cameras
        cv2.imshow('Entry Camera', frame_entry)
        cv2.imshow('Exit Camera', frame_exit)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap_entry.release()
    cap_exit.release()
    cv2.destroyAllWindows()

# Process frames for entry or exit events
def process_frame(frame, event_type, known_faces, known_names, attendance_data, last_entry_times):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = RetinaFace.detect_faces(frame)
    face_names = []
    for face_key in faces.keys():
        facial_area = faces[face_key]["facial_area"]
        top, right, bottom, left = facial_area[1], facial_area[2], facial_area[3], facial_area[0]
        encodings = face_recognition.face_encodings(frame_rgb, [(top, right, bottom, left)])

        for encoding in encodings:
            matches = face_recognition.compare_faces(known_faces, encoding, TOLERANCE)
            name = "Unknown"
            face_distances = face_recognition.face_distance(known_faces, encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_names[best_match_index]
            face_names.append(name)
    # Log detected names and timestamps
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    for name in face_names:
        if name != "Unknown":
            update_attendance(name, timestamp, event_type, attendance_data, last_entry_times)
            print(f"{event_type.capitalize()} detected for {name} at {timestamp}")
# Entry point
if __name__ == "__main__":
    known_faces, known_names = load_known_faces()
    recognize_faces_multi_camera(known_faces, known_names)



Loaded known faces.
Entry detected for vamshi at 2024-11-03 00:23:19
Updated attendance for vamshi for Period 6 (duration: 2.03 mins)
Exit detected for vamshi at 2024-11-03 00:25:21
